# Project №4


## Подготовка основы рекомендательной системы

### Задачи:
1. Познакомиться с датасетом, подготовить и проанализировать данные с помощью SQL;
2. Обработать данные средствами Python;
3. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить отчет продакт-менеджеру.


### Знакомство с данными:
- таблица carts с данными о пользовательских корзинах (дате создания, статусе, id пользователя-владельца и т. д.);
- таблица cart items с данными о курсах, которые пользователи добавили в корзину.


In [1]:
# Импортируем библиотеки
import psycopg2
import psycopg2.extras 
import pandas as pd
import numpy as np
import itertools as itt
from collections import Counter 

In [2]:
# Сколько клиентов покупали курсы
def getClientPurchasedCourses():
    query = '''select count(distinct user_id)
    from final.carts as fc
    join final.cart_items as fci on fc.id = fci.cart_id
    where fci.resource_type = 'Course'and fc.state = 'successful'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
client_purchased_courses = getClientPurchasedCourses()
print('За данный период ', client_purchased_courses[0]['count'], 'клиента купили курсы')


За данный период  49006 клиента купили курсы


In [3]:
# Сколько всего есть различных курсов
def getVariousCourses():
    query = '''select count(distinct resource_id)
    from final.carts as fc
    join final.cart_items as fci on fc.id = fci.cart_id
    where fci.resource_type = 'Course'
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
various_courses = getVariousCourses()
print('Всего в онлайн-школе ', various_courses[0]['count'], 'различных курсов')

Всего в онлайн-школе  127 различных курсов


In [4]:
# Каково среднее количество купленных курсов на одного клиенты
def getAverageAmountCourses():
    query = '''with orders as
    (select distinct fc.user_id, count(fci.resource_id) as cnt_orders
    from final.carts fc
    join final.cart_items fci on fc.id=fci.cart_id
    where fci.resource_type='Course' and fc.state = 'successful'
    group by 1)
    select sum(cnt_orders)/count(user_id)
    from orders
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
average_amount_courses = getAverageAmountCourses()
print('Среднее количество курсов на одного клиента составляет ', round(average_amount_courses[0]['?column?'],2))

Среднее количество курсов на одного клиента составляет  1.44


In [5]:
# Сколько клиентов купили больше одного курса
def getClientBuyMoreOneCourses():
    query = '''with clients as --CTE для определения пользователей, купивших курсы
    (select distinct fc.user_id, fci.resource_id
    from final.carts fc 
    join final.cart_items fci on fc.id=fci.cart_id
    where fc.state = 'successful' and fci.resource_type='Course'
    order by fc.user_id),
    orders as --CTE для подсчета количества курсов, купленных пользователем 
    (select user_id, count(resource_id) as cnt_course
    from clients
    group by 1)
    select count(user_id)
    from orders
    where cnt_course>1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
client_buy_more_one_courses = getClientBuyMoreOneCourses()
print(client_buy_more_one_courses[0]['count'], ' клиента купили более одного курса')

12656  клиента купили более одного курса


In [6]:
# Выгрузка данных по продажам курсов
def getDataUpload():
    query = '''with clients as --CTE для определения пользователей, купивших курсы
    (select distinct fc.user_id, fci.resource_id
    from final.carts fc 
    join final.cart_items fci on fc.id=fci.cart_id
    where fc.state = 'successful' and fci.resource_type='Course'
    order by fc.user_id),
    orders as --CTE для подсчета количества курсов, купленных пользователем
    (select user_id, count(resource_id) as cnt_course
    from clients
    group by 1)
    select o.user_id, o.cnt_course, fci.resource_id
    from orders o 
    join final.carts fc on o.user_id=fc.user_id
    join final.cart_items fci on fc.id=fci.cart_id
    where cnt_course>1 and 
    fc.state = 'successful' and fci.resource_type='Course'
    order by o.cnt_course desc, o.user_id  
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data
data_SQL = pd.DataFrame(getDataUpload())
data_SQL.head()

,user_id,cnt_course,resource_id
0,694146,19,566
1,694146,19,490
2,694146,19,564
3,694146,19,670
4,694146,19,907


In [7]:
# Читаем данные
df = pd.read_csv('data/SQL.csv')
print(df.head())

   user_id  resource_id  cnt_course
0   694146          566          19
1   694146          490          19
2   694146          564          19
3   694146          670          19
4   694146          907          19


In [8]:
# Группируем данные 
group_df = df.groupby('user_id')['resource_id'].apply(lambda x:list(np.unique(x))).reset_index()
print(group_df.head())

   user_id       resource_id
0       51       [516, 1099]
1     6117  [356, 357, 1125]
2    10275       [553, 1147]
3    10457       [361, 1138]
4    17166        [356, 357]


In [9]:
# Разбиваем на пары
list_courses = list()
for resource in group_df['resource_id']:
    for pair in itt.combinations(resource, 2):
        list_courses.append(pair)
# Преобразуем в множество, чтобы убрать повторы
list_courses_set = set(list_courses)
len(list_courses_set) #количество различных пар курсов встречаются вместе

3989

In [10]:
# Находим самую популярную пару курсов
popular = Counter(list_courses) #создаем словарь
popular = {k: v for k, v in sorted(popular.items(), key=lambda item: item[1], reverse=True)} #сортируем по значению
popular

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (504, 569): 139,
 (514, 515): 139,
 (551, 745): 138,
 (514, 566): 138,
 (502, 551): 135,
 (504, 840): 135,
 (571, 1125): 122,
 (502, 566): 120,
 (523, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (516, 553): 107,
 (551, 777): 107,
 (551, 679): 104,
 (356, 571): 103,
 (551, 564): 103,
 (515, 749): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

In [11]:
# Сортируем словарь по номеру курсов в паре (по ключу)
sorted_pair = {k: v for k, v in sorted(popular.items(), key=lambda item: item[0], reverse=False)}
sorted_pair

{(356, 357): 100,
 (356, 360): 1,
 (356, 361): 17,
 (356, 366): 15,
 (356, 367): 12,
 (356, 368): 1,
 (356, 489): 26,
 (356, 490): 13,
 (356, 502): 17,
 (356, 503): 1,
 (356, 508): 1,
 (356, 509): 5,
 (356, 513): 1,
 (356, 514): 35,
 (356, 515): 21,
 (356, 516): 16,
 (356, 517): 3,
 (356, 519): 14,
 (356, 523): 24,
 (356, 551): 48,
 (356, 552): 7,
 (356, 553): 5,
 (356, 564): 3,
 (356, 566): 21,
 (356, 568): 6,
 (356, 569): 2,
 (356, 570): 1,
 (356, 571): 103,
 (356, 659): 4,
 (356, 671): 2,
 (356, 672): 4,
 (356, 679): 8,
 (356, 742): 1,
 (356, 743): 1,
 (356, 745): 8,
 (356, 749): 1,
 (356, 750): 3,
 (356, 753): 3,
 (356, 756): 5,
 (356, 757): 1,
 (356, 764): 8,
 (356, 765): 35,
 (356, 776): 2,
 (356, 777): 4,
 (356, 791): 1,
 (356, 794): 10,
 (356, 800): 3,
 (356, 803): 1,
 (356, 809): 2,
 (356, 829): 5,
 (356, 835): 1,
 (356, 840): 1,
 (356, 862): 3,
 (356, 863): 1,
 (356, 866): 1,
 (356, 907): 1,
 (356, 908): 3,
 (356, 909): 3,
 (356, 912): 34,
 (356, 1099): 3,
 (356, 1100): 5,
 (

In [12]:
# Список уникальных курсов
courses = df['resource_id'].unique()

In [13]:
def recommend(course):
    course_list = [] #создаем пустой список
    for i in sorted_pair.keys(): #проходимся по ключам словаря
        if i[0] == course: #если первая пара из ключа равна значению курса
            course_list.append((i, sorted_pair[i])) #добавляем в созданный список саму пару и ее значение
    sorted_course_list = sorted(course_list, key=lambda x: x[1], reverse = True)
    return sorted_course_list[:2]

In [14]:
recomend_df = pd.DataFrame(columns = ['Recomandation 1', 'Recomandation 2']) #создаем датафрейм
for i in courses: #для каждого курса
    if len (recommend(i)) == 2: #применяем функцию и если выдает 2 рекомендации для курса
        recomend_df.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]] #записываем обе рекомендации
    elif len (recommend(i)) == 1: #применяем функцию и если выдает 1 рекомендацию для курса
        recomend_df.loc[i] = [recommend(i)[0][0][1], np.nan] #записываем одну рекомендацию, для второй ставим NaN
    else:
        recomend_df.loc[i] = [np.nan, np.nan] #если ничего не выдает ставим NaN

In [15]:
recomend_df

,Recomandation 1,Recomandation 2
566,794,570
490,566,551
564,566,809
670,756,764
907,1103,908
...,...,...
1201,NaN,NaN
833,NaN,NaN
365,551,566
902,NaN,NaN


In [16]:
recomend_df = recomend_df.fillna('-')
pd.set_option('display.max_rows', 150)
recomend_df

,Recomandation 1,Recomandation 2
566,794,570
490,566,551
564,566,809
670,756,764
907,1103,908
909,1100,1124
1102,1103,1115
1156,1181,1185
552,564,566
1103,1125,1115
